In [1]:
import pandas as pd
import sqlite3

## create a connection to the database

In [2]:
conn= sqlite3.connect('../data/checking-logs.sqlite')

## get the schema of the table test

In [3]:
pd.io.sql.read_sql('pragma table_info(test)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test

In [4]:
pd.io.sql.read_sql('select * from test limit 10', conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query:
do this by joining the table with the table deadlines;

the difference should be displayed in hours;

do not take the lab ’project1’ into account;

the value should be stored in the dataframe df_min with the corresponding uid;

## information about the deadlines table

In [5]:
pd.io.sql.read_sql('pragma table_info(deadlines)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [6]:
pd.io.sql.read_sql('select * from deadlines limit 10', conn)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [7]:
query_min = """
SELECT uid, MIN(diff) AS min_delta
FROM (
    SELECT 
        t.uid, 
        CAST((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM 
        test t
    LEFT JOIN 
        deadlines dl ON t.labname = dl.labs
    WHERE 
        t.labname != 'project1'
) AS subquery
ORDER BY min_delta
"""
query_max = """
SELECT uid, MAX(diff) AS max_delta
FROM (
    SELECT 
        t.uid, 
        CAST((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM 
        test t
    LEFT JOIN 
        deadlines dl ON t.labname = dl.labs
    WHERE 
        t.labname != 'project1'
) AS subquery
ORDER BY max_delta
"""
df_min = pd.read_sql(query_min, conn)
df_max = pd.read_sql(query_max, conn)

In [8]:
df_min


,uid,min_delta
0,user_30,-202


In [9]:
df_max

,uid,max_delta
0,user_25,-2


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg


In [10]:
query_avg ="""
SELECT AVG(diff) AS avg_delta
FROM ( SELECT uid, CAST((julianday(first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS diff
    FROM test
    LEFT JOIN deadlines dl on test.labname = dl.labs
    WHERE 
        test.labname != 'project1'
) AS subquery
ORDER BY avg_delta
"""

In [11]:
df_avg = pd.read_sql(query_avg, conn)
df_avg

,avg_delta
0,-89.125


## hypothesis: the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. we calculate the correlation coefficient between the number of pageviews and the difference

    using only one query, create a table with the columns: uid, avg_diff, pageviews
    uid is the uids that exist in the test
    avg_diff is the average delta between the first commit and the lab deadline per user
    pageviews is the number of Newsfeed visits per user
    do not take the lab ’project1’ into account
    store it to the dataframe views_diff
    use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference


In [12]:
query_views_diff = '''
SELECT
    t.uid,
    AVG(CAST((julianday(t.first_commit_ts) - julianday(dl.deadlines, 'unixepoch')) * 24 AS INTEGER)) AS avg_diff,
    (SELECT COUNT(*) FROM pageviews p WHERE p.uid = t.uid) AS pageviews
FROM
    test t
LEFT JOIN
    deadlines dl ON t.labname = dl.labs
LEFT JOIN 
    (SELECT uid, COUNT(*) AS pageviews FROM pageviews GROUP BY uid) pv ON t.uid = pv.uid
WHERE
    t.labname != 'project1'
GROUP BY
    t.uid;
'''

## use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [13]:
views_diff = pd.io.sql.read_sql(query_views_diff, conn)
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [14]:
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


## close the connection


In [15]:
conn.close()